In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from peft import PeftModel

# 模型和LoRA权重路径
model_path = '/root/autodl-tmp/finetuning/models/ChineseAlpacaGroup/llama-3-chinese-8b-instruct'
lora_path = './llama3_lora'

# 加载tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)

# 加载模型
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto", torch_dtype=torch.bfloat16)

# 加载LoRA权重
model = PeftModel.from_pretrained(model, model_id=lora_path, config=None)  # 如果有config参数，添加相应配置

# from peft import LoraConfig, TaskType, get_peft_model

# config = LoraConfig(
#     task_type=TaskType.CAUSAL_LM, 
#     target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
#     inference_mode=True, # 训练模式
#     r=8, # Lora 秩
#     lora_alpha=32, # Lora alaph，具体作用参见 Lora 原理
#     lora_dropout=0.1# Dropout 比例
# )


# 定义对话函数
def chat_with_model(prompt):
    messages = [
        {"role": "user", "content": prompt}
    ]
    
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    
    model_inputs = tokenizer([text], return_tensors="pt").to('cuda')
    
    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=512,
        do_sample=True,
        top_p=0.9,
        temperature=0.5,
        repetition_penalty=1.1,
        eos_token_id=tokenizer.encode('')[0],
        pad_token_id=tokenizer.eos_token_id # 不加这一行会出warning
    )
    
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return response
prompt = "孙答应和狂徒好上了"
print(chat_with_model(prompt))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

你怎么了？是不是有什么不对劲儿啊？


In [2]:

prompt = "你如何称呼我"
print(chat_with_model(prompt))

臣妾谨向皇上请安，皇上万福金安。


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from peft import PeftModel
# 对甄嬛模型进行推理
# 模型和LoRA权重路径
model_path = '/root/autodl-tmp/finetuning/llama3_zhenhuan'


# 加载tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)

# 加载模型
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto", torch_dtype=torch.bfloat16)

# 定义对话函数
def chat_with_model(prompt):
    messages = [
        {"role": "user", "content": prompt}
    ]
    
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    
    model_inputs = tokenizer([text], return_tensors="pt").to('cuda')
    
    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=512,
        do_sample=True,
        top_p=0.9,
        temperature=0.5,
        repetition_penalty=1.1,
        eos_token_id=tokenizer.encode('')[0],
        pad_token_id=tokenizer.eos_token_id # 不加这一行会出warning
    )
    
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return response
prompt = "你是谁，介绍一下你自己"
print(chat_with_model(prompt))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

我是甄嬛，家父是大理寺少卿甄远道。
